In [ ]:
# Получение данных
import pandas as pd

file_name = './Plus/data/plus729.csv'  # Откуда брать файл
data_source = pd.read_csv(file_name, sep=';', index_col=0,
                          skiprows=[1]  # Пропуск единиц измерения
                          ).dropna(
    axis=1, how='all')  # Чтение файла


task = ['max', 'min', 'min']
criteria_coeff = [-1 if s == 'max' else 1 for s in task]

column_names = data_source.columns.values  # Получение названий столбцов
# pd.options.display.float_format = '{:,.2f}'.format # Если нужно ограничить вывод до 2 знаков после запятой

# Разбиение таблицы на параметры и критерии
params = data_source.iloc[:, 0:6].astype(float)
criteria1_column = data_source.iloc[:, 6].astype(float) 
criteria2_column = data_source.iloc[:, 7].astype(float) 
criteria3_column = data_source.iloc[:, 8].astype(float)

print('Размер таблицы:', data_source.shape)

ERROR_VALUE = -32767
if data_source[data_source == ERROR_VALUE].count().sum() > 0 or data_source.isna().sum().sum()>0:
    print('В таблице есть битые данные')
    print(data_source[data_source == ERROR_VALUE].count())
    print(data_source.isna().sum())


In [ ]:
# Линейная интерполяция
from scipy.interpolate import LinearNDInterpolator
import numpy as np

# Создание точек для интерполяции
points = np.array(params.values)

PRINT_PROGRESS = False  # Вывод прогресса

X_LOWER = params.min().to_numpy()
X_UPPER = params.max().to_numpy()

# Создание интерполяторов
print('Расчет интерполяторов...')
print(f'Расчет интерполятора {column_names[6]}...')
criteria1 = LinearNDInterpolator(points, criteria1_column.values * criteria_coeff[0])
print(f'Расчет интерполятора {column_names[7]}...')
criteria2 = LinearNDInterpolator(points, criteria2_column.values * criteria_coeff[1])
print(f'Расчет интерполятора {column_names[8]}...')
criteria3 = LinearNDInterpolator(points, criteria3_column.values * criteria_coeff[2])
print('Расчет интерполяторов завершен')

sample_point = params.sample(1).values[0]
print('Расчет в случайной точке:',sample_point,criteria1(sample_point))

bounds = pd.DataFrame([X_LOWER, X_UPPER],
                      columns=column_names[0:6],
                      index=['Нижняя граница', 'Верхняя граница'])
bounds.transpose()


In [ ]:
# Однокритериальная оптимизация

import pandas as pd
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch  # Алгоритм Хука-Дживса
from pymoo.core.problem import Problem
from pymoo.optimize import minimize


class Find_best_c1 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = criteria1(x)


class Find_best_c2 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = criteria2(x)


class Find_best_c3 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = criteria3(x)


res_c1 = minimize(Find_best_c1(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)
res_c2 = minimize(Find_best_c2(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)
res_c3 = minimize(Find_best_c3(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)


# Сохранение результатов для однокритериальной оптимизации на основе маргинальных критериев
best_c1 = res_c1.F
best_c2 = res_c2.F
best_c3 = res_c3.F


# Вывод в виде таблицы
single_criteria_optimization_df = pd.DataFrame(
    columns=column_names[0:6],
    data=[res_c1.X, res_c2.X, res_c3.X],
    index=column_names[6:9]
)
single_criteria_optimization_df['Значение критерия'] = [
    criteria_coeff[0] * res_c1.F[0], criteria_coeff[1] * res_c2.F[0], criteria_coeff[2] * res_c3.F[0]]

print('Результаты однокритериальной оптимизации:')
single_criteria_optimization_df.transpose()


In [ ]:
# Оптимизация на основе маргинальных решений

from pymoo.optimize import minimize
from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch  # Алгоритм Хука-Дживса
import numpy as np


def marg_objective_function(x):
    '''Маргинальная целевая функция'''
    сurrent_c1 = criteria1(x)
    сurrent_c2 = criteria2(x)
    сurrent_c3 = criteria3(x)

    diff_c1 = best_c1 - сurrent_c1
    diff_c2 = сurrent_c2 - best_c2
    diff_c3 = сurrent_c3 - best_c3

    return np.sqrt(diff_c1**2 + diff_c2**2 + diff_c3**2)


# Постановка задачи однокритериальной оптимизации
class Marg_optimization(Problem):
    def __init__(self) -> None:
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = marg_objective_function(x)


# Запуск оптимизации
try:
    res_PS = minimize(
        Marg_optimization(), algorithm=PatternSearch(), seed=1)

    # Вывод результатов в виде таблицы
    margin_optimization_df = pd.DataFrame(
        data=(
            res_PS.X.tolist() + (criteria_coeff[0] * criteria1(res_PS.X)).tolist() + (criteria_coeff[1] * criteria2(res_PS.X)).tolist() + (criteria_coeff[2] * criteria3(res_PS.X)).tolist()),
        index=column_names[0:9],
    )
except Exception as e:
    print('Ошибка маргинальной оптимизации')
    print(e)

print('Результаты маргинальной оптимизации')
margin_optimization_df


In [ ]:
# Многокритериальная оптимизация
from pymoo.optimize import minimize
from pymoo.core.problem import Problem

# Алгоритмы оптимизации
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.age import AGEMOEA


# Постановка задачи
class Multi_objective_optimization(Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=3, n_constr=0,
                         # Минимальные значения параметров
                         xl=X_LOWER,
                         # Максимальные значения параметров
                         xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = criteria1(x)
        f2 = criteria2(x)
        f3 = criteria3(x)
        out["F"] = np.column_stack([f1, f2, f3])


problem = Multi_objective_optimization()

try:
    print('Многокритериальная оптимизация (Метод NSGA2)')
    res_NSGA2 = minimize(problem, algorithm=NSGA2(), seed=1,
                         verbose=PRINT_PROGRESS, termination=('n_gen', 50))  # Вывод прогресса)

    # Сохранение результатов
    res_NSGA2_df = pd.DataFrame(res_NSGA2.X)
    res_NSGA2_df['c1'] = criteria_coeff[0] * res_NSGA2.F[:, 0]
    res_NSGA2_df['c2'] = criteria_coeff[1] * res_NSGA2.F[:, 1]
    res_NSGA2_df['c3'] = criteria_coeff[2] * res_NSGA2.F[:, 2]
    res_NSGA2_df.columns = column_names
    res_NSGA2_df.to_csv('NSGA2_Results.csv', sep=',', index=False,
                        header=data_source.columns.values)

except Exception as e:
    print('Ошибка NSGA2')
    print(e)

try:
    print('Многокритериальная оптимизация (Метод AGEMOEA)')
    res_AGEMOEA = minimize(problem, algorithm=AGEMOEA(),
                           seed=1, verbose=PRINT_PROGRESS,
                           # time based termination
                           # Не позволяем наглеть
                           termination=('time', '00:01:00')
                           )

    # Сохранение результатов
    res_AGEMOEA_df = pd.DataFrame(res_AGEMOEA.X)
    res_AGEMOEA_df['c1'] = criteria_coeff[0] * res_AGEMOEA.F[:, 0]
    res_AGEMOEA_df['c2'] = criteria_coeff[1] * res_AGEMOEA.F[:, 1]
    res_AGEMOEA_df['c3'] = criteria_coeff[2] * res_AGEMOEA.F[:, 2]
    res_AGEMOEA_df.columns = column_names
    res_AGEMOEA_df.to_csv('AGEMOEA_Results.csv', sep=',', index=False,
                          header=data_source.columns.values)
except Exception as e:
    print('Ошибка AGEMOEA')
    print(e)


In [ ]:
res_NSGA2_df


In [ ]:
res_AGEMOEA_df